In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
import os
import  pickle
import h5py
import json

class CustomImageDataset(Dataset):
    def __init__(self, mode = "train"):
        if mode == "train":
            with open("./train.txt", "r") as f:
                self.datalist = f.readlines()
        elif mode == "test":
            with open("./test.txt", "r") as f:
                self.datalist = f.readlines()
        text_tmp = open("data.json")
        data = json.load(text_tmp)
        ann = {"假": 0, "真": 1, "辟谣": 2}
        self.label_dict = {}
        for i in data:
            self.label_dict[i["video_id"]] = ann[i["annotation"]]

    def __len__(self):
        return len(self.datalist)

    def __getitem__(self, idx):
        imf = open('/mnt/c/Users/IT/Downloads/all/ptvgg19_frames/'+self.datalist[idx].strip()+'.pkl', 'rb')
        data_imf = pickle.load(imf)
        vf = h5py.File('/mnt/c/Users/IT/Downloads/all/c3d/'+self.datalist[idx].strip()+'.hdf5', 'r')
        data_vf = vf[self.datalist[idx].strip()]["c3d_features"][:]
        af = open('/mnt/c/Users/IT/Downloads/all/audiosconvfea/'+self.datalist[idx].strip(), 'rb')
        data_af = pickle.load(af)
        label = self.label_dict[self.datalist[idx].strip()]

        return data_imf, data_vf, data_af, label

In [5]:
c = CustomImageDataset()

In [6]:
data_imf, data_vf, data_af, label = c[10]

In [7]:
data_imf.shape, data_vf.shape, data_af.shape

((28, 4096), (28, 4096), (20, 12288))

In [60]:
data_imf

array([[-0.02978136,  0.5214211 , -2.2010493 , ..., -0.25699484,
        -0.8722791 , -1.8240757 ],
       [ 0.31891212,  0.20015962, -2.284183  , ..., -0.47997105,
        -0.7861618 , -2.415896  ],
       [ 0.67959756,  0.6448401 , -2.2429893 , ..., -0.54838455,
        -0.8340438 , -1.8753573 ],
       ...,
       [-0.66805995,  0.32872668, -0.6851071 , ..., -0.2504583 ,
        -0.45006207,  0.2953867 ],
       [-0.6856191 ,  0.3228754 , -0.6766888 , ..., -0.24488266,
        -0.43595433,  0.3240003 ],
       [-0.70799834,  0.31297606, -0.69428504, ..., -0.23087285,
        -0.4357256 ,  0.33114514]], dtype=float32)

In [61]:
data_vf

array([[0.21621627, 1.2825677 , 1.447454  , ..., 0.        , 1.7883198 ,
        0.12972558],
       [0.        , 1.1042053 , 0.47129965, ..., 0.        , 0.        ,
        0.7814119 ],
       [0.        , 0.00869048, 0.        , ..., 0.        , 0.        ,
        1.7861747 ],
       ...,
       [0.        , 0.        , 2.665276  , ..., 0.55926245, 0.        ,
        0.        ],
       [0.        , 0.        , 2.6870937 , ..., 0.95208526, 0.        ,
        0.        ],
       [0.        , 0.        , 2.6269674 , ..., 1.0240164 , 0.        ,
        0.        ]], dtype=float32)

text

In [20]:
keywords = []

with open('data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

for i in range(len(data)):
    keywords.append(data[i]["keywords"])
keywords

['100多颗恒星“离奇消失”是被外星人操控了',
 '100多颗恒星“离奇消失”是被外星人操控了',
 '100多颗恒星“离奇消失”是被外星人操控了',
 '100多颗恒星“离奇消失”是被外星人操控了',
 '100多颗恒星“离奇消失”是被外星人操控了',
 '119徐州化工厂爆炸',
 '119徐州化工厂爆炸',
 '119徐州化工厂爆炸',
 '120被堵路口致患者抢救无效死亡',
 '120被堵路口致患者抢救无效死亡',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '16名韩国护士因疫情严重辞职',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '19岁消防员救火时被坍塌的墙体压倒而牺牲',
 '2019年8月一架客机在黑龙江失事，42人遇难',
 '2019年8月一架客机在黑龙江失事，42人遇难',
 '2019年8月一架客机在黑龙江失事，42人遇难',
 '2019年8月一架客机在黑龙江失事，42人遇难',
 '2019年8月一架客机在黑龙江失事，42人遇难',
 '2019环邢台国际自行车赛连环大撞车',
 '2019环邢台国际自行车赛连环大撞车',
 '2019环邢台国际自行车赛连环大撞车',
 '2019环邢台国际自行车赛连环大撞车',
 '2019环邢台国际自行车赛连环大撞车',


In [23]:
import torch
from transformers import BertTokenizer, BertModel

model_name = './bert-base-chinese/'
tokenizer = BertTokenizer.from_pretrained(model_name)

inputs = tokenizer(keywords, return_tensors='pt', padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [25]:
from transformers import BertModel, BertTokenizer

model_name = './bert-base-chinese/'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 使用模型和分词器进行文本处理和编码
text = "Hello, how are you?"
tokens = tokenizer.tokenize(text)
input_ids = tokenizer.convert_tokens_to_ids(tokens)

print(tokens)
print(input_ids)


['hello', ',', 'how', 'are', 'you', '?']
[8701, 117, 9510, 8995, 8357, 136]


In [26]:



model = BertModel.from_pretrained(model_name)


# 获取BERT模型的输出
with torch.no_grad():
    outputs = model(**inputs)

# 获取特征
last_hidden_states = outputs.last_hidden_state
features = last_hidden_states.mean(dim=1)  # 对所有token的表示取平均作为句子表示

# 特征现在包含了BERT对关键词的表示
print(features)


tensor([[ 0.1991, -0.0138, -0.1109,  ...,  0.2633,  0.0440, -0.0900],
        [ 0.1991, -0.0138, -0.1109,  ...,  0.2633,  0.0440, -0.0900],
        [ 0.1991, -0.0138, -0.1109,  ...,  0.2633,  0.0440, -0.0900],
        ...,
        [-0.2289,  0.5408, -0.4242,  ..., -0.0441, -0.1671, -0.2258],
        [-0.2289,  0.5408, -0.4242,  ..., -0.0441, -0.1671, -0.2258],
        [-0.2289,  0.5408, -0.4242,  ..., -0.0441, -0.1671, -0.2258]])


In [27]:
features.shape

torch.Size([5495, 768])

In [33]:
features_path = './datas/text_f.pt'

In [34]:
torch.save(features, features_path)